In [1]:
import os
import caesar
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import modules as anal
import modules.anal_func as anal_func

from astropy.cosmology import LambdaCDM
import astropy.units as u
from astropy.io import fits
from scipy import interpolate

from modules.powdermod.makesed import MakeSED

cosmo = LambdaCDM(H0 = 68, Om0= 0.3, Ode0= 0.7, Ob0=0.048)
plt.rcParams.update({'font.size': 28})

sb = anal.Simba('PC39BP4', size=25)

In [2]:
# select the galaxies for the SED

In [3]:
# initialize SED making class
hydro_dir_base = '/home/lorenzong/analize_simba_cgm/output/hdf5/filtered_files/'
preselect = '/home/lorenzong/analize_simba_cgm/output/txt/target_selection_for_SED/test_selection.txt'
selection_file = 'test_entire_sfh'
makesed = MakeSED(sb, nnodes=1, model_run_name='m25', hydro_dir_base=hydro_dir_base, selection_file=selection_file, preselect=preselect)

In [6]:
makesed.selection_gals([129], [1]) # create the hdf5 selection
makesed.create_master('local') # create the parameters masters

yt : [INFO     ] 2024-08-21 18:43:30,512 Opening /media/lorenzong/Data1/simba_hig_res/m25n512_129.hdf5
yt : [INFO     ] 2024-08-21 18:43:30,529 Found 263961 halos
yt : [INFO     ] 2024-08-21 18:43:30,536 Found 5290 galaxies


Inserted aggregated data for: snap129
===================== snap129/code_coods [[17192.895 11200.677 13909.615]]
Processing galaxy:  1  in snap  129
Initializing position:  [17192.895 11200.677 13909.615]
Generating local run script
/home/lorenzong/analize_simba_cgm/output/hdf5/powderday_sed_out/snap_129/run_local.sh
Executing local run script...
Starting simulation...


yt : [INFO     ] 2024-08-21 18:43:32,165 Calculating time from 7.114e-01 to be 2.969e+17 seconds
yt : [INFO     ] 2024-08-21 18:43:32,165 Assuming length units are in kpc/h (comoving)
yt : [INFO     ] 2024-08-21 18:43:32,201 Parameters: current_time              = 2.968791388457766e+17 s
yt : [INFO     ] 2024-08-21 18:43:32,201 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2024-08-21 18:43:32,201 Parameters: domain_left_edge          = [-120. -120. -120.]
yt : [INFO     ] 2024-08-21 18:43:32,201 Parameters: domain_right_edge         = [120. 120. 120.]
yt : [INFO     ] 2024-08-21 18:43:32,201 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2024-08-21 18:43:32,201 Parameters: current_redshift          = 0.4057579856290421
yt : [INFO     ] 2024-08-21 18:43:32,201 Parameters: omega_lambda              = 0.7
yt : [INFO     ] 2024-08-21 18:43:32,201 Parameters: omega_matter              = 0.3
yt : [INFO     ] 2024-08-21 18:43:32,201 Parameters: omega_radiation  

Simulation complete


In [4]:
makesed.plotsed(129, 1, show=True)

saving


AttributeError: 'Axes' object has no attribute 'show'